**Autor:** Jose Ricardo Mendoza Villar  
371014 Limpieza y Manipulación de Datos  
Primer examen parcial

***
# General

In [ ]:
HOME_PATH = "data/Examen"

In [ ]:
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.gofplots import qqplot

In [ ]:
def Zcore_outlier(df):
    outliers = []
    mean = np.mean(df)
    sd = np.std(df)
    for i in df:
        z = (i - mean) / sd
        if np.abs(z) > 3:
            outliers.append(i)
    return outliers

In [ ]:
def box_plot(df):
    plt.title("Box plot")
    sns.boxplot(df)
    plt.show()

In [ ]:
def qqplots(df):
    qqplot(df, line='s')
    plt.title("Normal QQPlot")
    plt.show()

In [ ]:
def levenshteinDistance(token1, token2):
    distances = np.zeros([len(token1) + 1, len(token2) + 1])

    for t1 in range(len(token1) + 1):
        distances[t1][0] = t1
    for t2 in range(len(token2) + 1):
        distances[0][t2] = t2

    a = 0
    b = 0
    c = 0
    for t1 in range(1, len(token1) + 1):
        for t2 in range(1, len(token2) + 1):
            if (token1[t1 - 1] == token2[t2 - 1]):
                distances[t1][t2] = distances[t1 - 1][t2 - 1]
            else:
                a = distances[t1][t2 - 1]
                b = distances[t1 - 1][t2]
                c = distances[t1 - 1][t2 - 1]
                if (a <= b and a <= c):
                    distances[t1][t2] = a + 1
                elif (b <= a and b <= c):
                    distances[t1][t2] = b + 1
                else:
                    distances[t1][t2] = c + 1
    return distances[len(token1)][len(token2)]

***
# Hipertensión arterial de población en México

In [ ]:
df = pd.read_csv(f"{HOME_PATH}/Hipertension_Arterial_Mexico.csv")

In [ ]:
# ¿Cuántas variables contiene la base?
len(df.columns)

In [ ]:
# ¿Cuántos registros contiene la base?
len(df)

In [ ]:
# Realiza el histograma de la concentración de triglicéridos y sube la imagen o captura de pantalla.
plt.hist(df["valor_trigliceridos"], bins=15, linewidth = 0.5, edgecolor="White")
plt.title('Concentración de triglicéridos')
plt.xlabel('Concentración de triglicéridos')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
# Respecto a la pregunta anterior, ¿existen datos atípicos? ¿cuáles son y por qué?
print(Zcore_outlier(df["valor_trigliceridos"]))
print()
print(min(Zcore_outlier(df["valor_trigliceridos"])))

In [ ]:
box_plot(df["valor_trigliceridos"])

In [ ]:
qqplots(df["valor_trigliceridos"])

In [ ]:
# Selecciona la variable "Tensión arterial" y busca aquellos datos menores de 75, ¿son datos atípicos? ¿por qué?
atipicos = np.array(Zcore_outlier(df["tension_arterial"]))
atipicos[atipicos < 75]

In [ ]:
box_plot(df["tension_arterial"])

***
# COVID en Brazil

In [ ]:
covid_brazil = pd.read_csv(f"{HOME_PATH}/brazil_covid19_old.csv")

In [ ]:
covid_brazil.shape, covid_brazil[covid_brazil.duplicated(["date", "hour", "state"],keep=False)].sort_values(by=["date", "hour", "state"]).shape

In [ ]:
# Busca datos duplicados respecto a las variables fecha y hora y comparte una captura de pantalla de ellos.
covid_brazil[covid_brazil.duplicated(["date", "hour", "state"],keep=False)].sort_values(by=["date", "hour", "state"])

In [ ]:
# ¿Cuál es la distancia de Levenstein entre las frases 'Laura McGille rivestein' y 'Loura MCGile Levenstein'?
s1 = 'Laura McGille rivestein'
s2 = 'Loura MCGile Levenstein'
nltk.edit_distance(s1,s2), levenshteinDistance(s1,s2)

***
# Terrorismo global 

In [ ]:
terrorismo_global = pd.read_csv(f"{HOME_PATH}/globalterrorismdb_0718dist.csv", encoding="ISO-8859-1", low_memory=False)

In [ ]:
# ¿Cuál de las siguientes variables tiene más del 25% de valores faltantes?
for col in ["INT_ANY", "approxdate", "INT_MISC", "INT_LOG"]:
    print(col, terrorismo_global[col].isnull().sum() / terrorismo_global.shape[0] * 100)

In [ ]:
#Elimina las columnas que tienen más del 25% de valores perdidos, 
# luego crea un mapa de calor para datos perdidos
mas_25_porciento = [col for col in terrorismo_global.columns if (terrorismo_global[col].isnull().sum() / terrorismo_global.shape[0]) > 0.25]
terrorismo_global_reduce = terrorismo_global.drop(mas_25_porciento, axis=1)

plt.figure(figsize=(16, 6))
sns.heatmap(terrorismo_global_reduce.isnull(), cbar=False, cmap='viridis')
plt.show()

In [ ]:
# Elimina todos los registros que tienen datos faltantes de la base original, ¿cuántos registros quedan en total?
terrorismo_global.dropna().shape[0]